In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from concurrent.futures import ProcessPoolExecutor
import time
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime


In [ ]:
# from concurrent.futures import ThreadPoolExecutor  
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.firefox.service import Service
# from selenium.webdriver.firefox.options import Options
# from selenium.common.exceptions import NoSuchElementException, TimeoutException
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# import time
# import os
# import pandas as pd
# import requests
# from bs4 import BeautifulSoup
# from datetime import datetime

# # 🔹 Function to Start Firefox Browser
# def start_browser(link, dfolder, geko_path, window_size='1500,1080'):
#     os.makedirs(dfolder, exist_ok=True)
#     options = Options()
#     options.add_argument('--start-maximized')
#     options.set_preference('privacy.trackingprotection.enabled', True)
#     service = Service(geko_path)
#     browser = webdriver.Firefox(service=service, options=options)
    
#     width, height = map(int, window_size.split(','))
#     browser.set_window_size(width, height)
    
#     browser.get(link)
#     time.sleep(3)  # Let page load
    
#     return browser

# # 🔹 Function to Search, Scroll & Scrape Hotels
# def search_and_scrape(browser, city, year, month, month_text, arrival_day, departure_day, filename):
#     try:
#         print(f"Searching for {city} ({year}-{month}-{arrival_day} to {year}-{month}-{departure_day})")

#         # Click search bar and enter city
#         WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id=":rh:"]'))).click()
#         time.sleep(1)
#         search_box = WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id=":rh:"]')))
#         search_box.send_keys(city)
#         time.sleep(1)
#         browser.find_element(By.XPATH, '//*[@class="ebbedaf8ac ab26a5d2bd e33c97ff6b"]').click()
#         time.sleep(2)

#         # Select correct month and year
#         target_date = month_text + ' ' + year
#         while browser.find_element(By.XPATH, '//h3[contains(@class, "e1eebb6a1e ee7ec6b631")]').text.strip() != target_date:
#             browser.find_element(By.XPATH, '//*[@class="a83ed08757 c21c56c305 f38b6daa18 d691166b09 f671049264 f4552b6561 dc72a8413c f073249358"]').click()
#             time.sleep(1)

#         # Select arrival and departure dates
#         dates = browser.find_elements(By.XPATH, '//table[@class="eb03f3f27f"]//td[@class="b80d5adb18"]//span[@class="cf06f772fa ef091eb985"]')
#         for date in dates:
#             if date.get_attribute("data-date") == f"{year}-{month}-{arrival_day}":
#                 date.click()
#             if date.get_attribute("data-date") == f"{year}-{month}-{departure_day}":
#                 date.click()
#                 break
#         time.sleep(2)

#         # Click search button
#         browser.find_element(By.XPATH, '//button[@class="a83ed08757 c21c56c305 a4c1805887 f671049264 a2abacf76b c082d89982 cceeb8986b b9fd3c6b3c"]').click()
#         time.sleep(2)

#         # 🔹 Scroll and load all results
#         print(f"Scrolling and clicking 'See More' for {city}...")
#         i = 0
#         while True:
#             try:
#                 total_height = browser.execute_script("return document.body.scrollHeight")
#                 browser.execute_script(f"window.scrollTo(0, {total_height});")
#                 time.sleep(2)  # Allow content to load

#                 load_more_button = WebDriverWait(browser, 5).until(
#                     EC.element_to_be_clickable((By.XPATH, '//button[@class="a83ed08757 c21c56c305 bf0537ecb5 f671049264 af7297d90d c0e0affd09"]'))
#                 )
#                 load_more_button.click()
#                 i += 1
#             except (NoSuchElementException, TimeoutException):
#                 break
        
#         print(f'🔹 The "See More" button was clicked {i} times')

#         # **FIXED: Ensure browser stays open for scraping**
#         print(f"Scraping hotels for {city}...")
#         hotel_list_data = scrape_hotel_list_page_from_selenium(browser)
        
#         # 🔹 Handle Empty Scraping Result (Retry Once)
#         if not hotel_list_data:
#             print("⚠ No hotels found. Retrying...")
#             time.sleep(3)  # Allow page to fully load
#             hotel_list_data = scrape_hotel_list_page_from_selenium(browser)

#         if hotel_list_data:
#             df = pd.DataFrame(hotel_list_data, columns=['Name', 'Price', 'Rating', 'Detail Link'])
#             df.to_csv(filename, index=False)
#             print(f"✅ Hotel data saved to {filename}")
#         else:
#             print("❌ Failed to scrape any hotels.")

#     except Exception as e:
#         print(f"❌ Error during search and scrape: {e}")

#     finally:
#         print(f"🛑 Closing browser for {city}")
#         browser.quit()

# # 🔹 Function to Scrape Hotel Data
# def scrape_hotel_list_page_from_selenium(browser):
#     page_source = browser.page_source
#     soup = BeautifulSoup(page_source, 'html.parser')
    
#     hotel_data = []
#     hotels = soup.find_all('div', {'data-testid': 'property-card'})
    
#     for hotel in hotels:
#         name = hotel.find('div', {'data-testid': 'title'}).get_text(strip=True) if hotel.find('div', {'data-testid': 'title'}) else "NA"
#         price = hotel.find('span', {'class': 'f6431b446c fbfd7c1165 e84eb96b1f'}).get_text(strip=True) if hotel.find('span', {'class': 'f6431b446c fbfd7c1165 e84eb96b1f'}) else "NA"
#         rating_container = hotel.find('div', {'data-testid': 'review-score'})
#         rating = rating_container.find('div').get_text(strip=True) if rating_container else "NA"
#         detail_link = hotel.find('a', {'data-testid': 'title-link'})['href'] if hotel.find('a', {'data-testid': 'title-link'}) else None 
#         if detail_link:
#             detail_link = f"https://www.booking.com{detail_link}"
        
#         hotel_data.append([name, price, rating, detail_link])
    
#     return hotel_data

# # 🔹 Function to Handle Multiple Browsers
# def scrape_with_browser(browser_id):
#     dfolder = './downloads'
#     geko_path = ''
#     link = 'https://www.booking.com/index.es.html'
    
#     browser = start_browser(link, dfolder, geko_path)

#     locations = [
#         ("Barcelona", "2025", "03", "marzo", "01", "07", "hotels_barcelona_MWC.csv"),
#         ("Madrid", "2025", "03", "marzo", "01", "07", "hotels_madrid_mar_MWC.csv"),
#         ("Barcelona", "2025", "03", "marzo", "22", "28", "hotels_barcelona_after_MWC.csv"),
#         ("Madrid", "2025", "03", "marzo", "22", "28", "hotels_madrid_after_MWC.csv")
#     ]

#     city, year, month, month_text, arrival_day, departure_day, filename = locations[browser_id]
#     search_and_scrape(browser, city, year, month, month_text, arrival_day, departure_day, filename)

# # 🔹 Run 4 Browsers in Parallel
# if __name__ == "__main__":
#     num_browsers = 4
#     with ThreadPoolExecutor(max_workers=num_browsers) as executor:
#         executor.map(scrape_with_browser, range(num_browsers))


In [4]:
 from concurrent.futures import ThreadPoolExecutor  
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime

# 🔹 Function to Start Firefox Browser
def start_browser(link, dfolder, geko_path, window_size='1500,1080'):
    os.makedirs(dfolder, exist_ok=True)
    options = Options()
    options.add_argument('--start-maximized')
    options.set_preference('privacy.trackingprotection.enabled', True)
    service = Service(geko_path)
    browser = webdriver.Firefox(service=service, options=options)
    
    width, height = map(int, window_size.split(','))
    browser.set_window_size(width, height)
    
    browser.get(link)
    time.sleep(3)  # Let page load
    
    return browser

# 🔹 Function to Search, Scroll & Scrape Hotels
def search_and_scrape(browser, city, year, month, month_text, arrival_day, departure_day, filename):
    try:
        print(f"Searching for {city} ({year}-{month}-{arrival_day} to {year}-{month}-{departure_day})")

        # Click search bar and enter city
        WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id=":rh:"]'))).click()
        time.sleep(1)
        search_box = WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id=":rh:"]')))
        search_box.send_keys(city)
        time.sleep(1)
        browser.find_element(By.XPATH, '//*[@class="ebbedaf8ac ab26a5d2bd e33c97ff6b"]').click()
        time.sleep(2)

        # Select correct month and year
        target_date = month_text + ' ' + year
        while browser.find_element(By.XPATH, '//h3[contains(@class, "e1eebb6a1e ee7ec6b631")]').text.strip() != target_date:
            browser.find_element(By.XPATH, '//*[@class="a83ed08757 c21c56c305 f38b6daa18 d691166b09 f671049264 f4552b6561 dc72a8413c f073249358"]').click()
            time.sleep(1)

        # Select arrival and departure dates
        dates = browser.find_elements(By.XPATH, '//table[@class="eb03f3f27f"]//td[@class="b80d5adb18"]//span[@class="cf06f772fa ef091eb985"]')
        for date in dates:
            if date.get_attribute("data-date") == f"{year}-{month}-{arrival_day}":
                date.click()
            if date.get_attribute("data-date") == f"{year}-{month}-{departure_day}":
                date.click()
                break
        time.sleep(2)

        # Click search button
        browser.find_element(By.XPATH, '//button[@class="a83ed08757 c21c56c305 a4c1805887 f671049264 a2abacf76b c082d89982 cceeb8986b b9fd3c6b3c"]').click()
        time.sleep(2)

        # 🔹 Scroll and load all results
        print(f"Scrolling and clicking 'See More' for {city}...")
        i = 0
        while True:
            try:
                total_height = browser.execute_script("return document.body.scrollHeight")
                browser.execute_script(f"window.scrollTo(0, {total_height});")
                time.sleep(2)  # Allow content to load

                load_more_button = WebDriverWait(browser, 5).until(
                    EC.element_to_be_clickable((By.XPATH, '//button[@class="a83ed08757 c21c56c305 bf0537ecb5 f671049264 af7297d90d c0e0affd09"]'))
                )
                load_more_button.click()
                i += 1
            except (NoSuchElementException, TimeoutException):
                break
        
        print(f'🔹 The "See More" button was clicked {i} times')

        # Scrape hotels
        print(f"Scraping hotels for {city}...")
        hotel_list_data = scrape_hotel_list_page_from_selenium(browser)

        # **Extract descriptions**
        descriptions = []
        for hotel in hotel_list_data:
            if hotel[3] != "NA":
                descriptions.append(scrape_hotel_detail_page(hotel[3]))  # Append description
            else:
                descriptions.append("NA")  # No link, add "NA" to description
        
        # Convert to DataFrame
        if hotel_list_data:
            df = pd.DataFrame(hotel_list_data, columns=['Name', 'Price', 'Rating', 'Detail Link'])
            df['Description'] = descriptions  # Add descriptions column
            df.to_csv(filename, index=False)
            print(f"✅ Hotel data saved to {filename}")
        else:
            print("❌ No hotels found.")

    except Exception as e:
        print(f"❌ Error during search and scrape: {e}")

    finally:
        print(f"🛑 Closing browser for {city}")
        browser.quit()

# 🔹 Function to Scrape Hotel List
def scrape_hotel_list_page_from_selenium(browser):
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    
    hotel_data = []
    hotels = soup.find_all('div', {'data-testid': 'property-card'})
    
    for hotel in hotels:
        name = hotel.find('div', {'data-testid': 'title'}).get_text(strip=True) if hotel.find('div', {'data-testid': 'title'}) else "NA"
        price = hotel.find('span', {'class': 'f6431b446c fbfd7c1165 e84eb96b1f'}).get_text(strip=True) if hotel.find('span', {'class': 'f6431b446c fbfd7c1165 e84eb96b1f'}) else "NA"
        rating_container = hotel.find('div', {'data-testid': 'review-score'})
        rating = rating_container.find('div').get_text(strip=True) if rating_container else "NA"
        detail_link = hotel.find('a', {'data-testid': 'title-link'})['href'] if hotel.find('a', {'data-testid': 'title-link'}) else None
        detail_link = detail_link if detail_link else "NA"
        
        hotel_data.append([name, price, rating, detail_link])
    
    return hotel_data

# 🔹 Function to Scrape Hotel Description
def scrape_hotel_detail_page(detail_url):
    headers = {'User-Agent': 'Mozilla/5.0'}
    
    try:
        response = requests.get(detail_url, headers=headers, timeout=10)
        if response.status_code != 200:
            return "Failed to load page"

        soup = BeautifulSoup(response.text, 'html.parser')
        
        # **Try multiple possible description locations**
        description = soup.find('p', {'data-testid': 'property-description'})
        if not description:
            description = soup.find('div', {'class': 'a53cbfa6de b3efd73f69'})
        if not description:
            description = soup.find('span', {'class': 'e84eb96b1f'})
        
        return description.get_text(strip=True) if description else "NA"
    
    except Exception as e:
        print(f"Error scraping details from {detail_url}: {e}")
        return "Error"
def scrape_with_browser(browser_id):
    dfolder = './downloads'
    geko_path = ''  # Add the correct GeckoDriver path
    link = 'https://www.booking.com/index.es.html'
    
    browser = start_browser(link, dfolder, geko_path)

    locations = [
        ("Barcelona", "2025", "03", "marzo", "01", "07", "hotels_barcelona_MWC.csv"),
        ("Madrid", "2025", "03", "marzo", "01", "07", "hotels_madrid_MWC.csv"),
        ("Barcelona", "2025", "03", "marzo", "22", "28", "hotels_barcelona_after_MWC.csv"),
        ("Madrid", "2025", "03", "marzo", "22", "28", "hotels_madrid_after_MWC.csv")
    ]

    city, year, month, month_text, arrival_day, departure_day, filename = locations[browser_id]
    search_and_scrape(browser, city, year, month, month_text, arrival_day, departure_day, filename)
if __name__ == "__main__":
    with ThreadPoolExecutor(max_workers=4) as executor:
        executor.map(scrape_with_browser, range(4))

Searching for Barcelona (2025-03-22 to 2025-03-28)
Searching for Barcelona (2025-03-01 to 2025-03-07)
Searching for Madrid (2025-03-01 to 2025-03-07)
Searching for Madrid (2025-03-22 to 2025-03-28)
Scrolling and clicking 'See More' for Barcelona...
Scrolling and clicking 'See More' for Barcelona...
Scrolling and clicking 'See More' for Madrid...
Scrolling and clicking 'See More' for Madrid...
🔹 The "See More" button was clicked 0 times
Scraping hotels for Barcelona...
🔹 The "See More" button was clicked 0 times
Scraping hotels for Madrid...
✅ Hotel data saved to hotels_barcelona_after_MWC.csv
🛑 Closing browser for Barcelona
✅ Hotel data saved to hotels_madrid_MWC.csv
🛑 Closing browser for Madrid
🔹 The "See More" button was clicked 37 times
Scraping hotels for Madrid...
🔹 The "See More" button was clicked 37 times
Scraping hotels for Barcelona...
Error scraping details from https://www.booking.com/hotel/es/spacious-and-cozy-caribbean-boat-in-barcelona.es.html?label=gen173nr-1FCAEoggI46A